In [16]:
from feat_competition_sales import *
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import time
import xgboost as xgb

In [2]:
# Tell plotly to plot in a notebook
offline.init_notebook_mode(connected=True)

### Load and plot data

In [3]:
# Load data
store = pd.read_csv('data/store.csv')
train = pd.read_csv('data/train.csv', dtype = {'StateHoliday': np.str})

# Merge
train = pd.merge(store, train, on='Store')
train = date_convert(train)

# Split
end_val = np.floor(0.8 * train.shape[0]).astype(int)
end_test = np.floor(0.9 * train.shape[0]).astype(int)

Train = train.loc[:end_val]
Val = train.loc[end_val:end_test]
Test = train.loc[end_test:]

In [4]:
#plot_hists(store, name='store')

In [5]:
#plot_hists(train, name='train')

### Feature engineering 

In [6]:
# Clean data, construct new features for xgboost
Train = data_transformation(Train)
Val = data_transformation(Val)
Test = data_transformation(Test)

/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning:

divide by zero encountered in log

/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning:

invalid value encountered in log



In [7]:
# OHE
cols = Train.select_dtypes(include='object').columns.tolist()

for col in cols:
    Train[col] = Train[col].astype(str)
    Val[col] = Val[col].astype(str)
    Test[col] = Test[col].astype(str)

ohe = OneHotEncoder(handle_unknown='ignore')
ohe_train = pd.DataFrame()
ohe_val = pd.DataFrame()
ohe_test = pd.DataFrame()


for col in cols:
    ohe.fit(np.array(Train[col]).reshape(-1,1))
    ohe_train_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Train[col]).reshape(-1,1)).toarray())
    ohe_val_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Val[col]).reshape(-1,1)).toarray()) 
    ohe_test_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Test[col]).reshape(-1,1)).toarray()) 
    ohe_train = pd.concat([ohe_train, ohe_train_tmp], axis=1)
    ohe_val = pd.concat([ohe_val, ohe_train_tmp], axis=1)
    ohe_test = pd.concat([ohe_test, ohe_train_tmp], axis=1)

In [8]:
# Drop columns
Train.drop(axis=1, labels=cols, inplace=True)
Val.drop(axis=1, labels=cols, inplace=True)
Test.drop(axis=1, labels=cols, inplace=True)

In [9]:
# Concat
Train = pd.concat([Train.reset_index(), ohe_train], axis=1)
Val = pd.concat([Val.reset_index(), ohe_train], axis=1)
Test = pd.concat([Test.reset_index(), ohe_train], axis=1)

In [11]:
# Write data to csv to reuse it fast
#Train.to_csv('data/Train.csv', index=False)
#Test.to_csv('data/Test.csv', index=False)
#Val.to_csv('data/Val.csv', index=False)

In [12]:
Train = pd.read_csv('data/Train.csv')
Val = pd.read_csv('data/Val.csv')
Test = pd.read_csv('data/Test.csv')

In [14]:
# Create X_train, y_train and so on
# XGB
X_train = Train.drop(axis=1, labels='Sales')
X_train = X_train
y_train = Train['Sales']
X_val = Val.drop(axis=1, labels='Sales')
y_val = Val['Sales']
X_test = Test.drop(axis=1, labels='Sales')
y_test = Test['Sales']

### Training

In [146]:
def parameter_search(X, y, method, params, steps=None):
    valid = {'elastic', 'rf', 'xgboost', 'stack'}
    if method not in valid:
        raise ValueError("Method must be one of %r." % valid)
        
    if method == 'elastic':
        if not type(params) == dict:
            raise ValueError('params is not a dictionary, please support a dictionary of parameters')
        else:
            score = []
            end = np.floor(0.8 * X.shape[0]).astype(int)
            X_train = np.array(X)[:end]
            y_train = np.array(y)[:end]
            X_cv = np.array(X)[end:]
            y_cv = np.array(y)[end:]

        # Grid search for elastic net
        for ratio in params['ratio']:
            for alpha in params['alpha']:
                elastic = ElasticNet(alpha=alpha, l1_ratio=ratio)
                elastic.fit(X_train, y_train)
                preds = elastic.predict(X_cv)                                       
                score.append((alpha, ratio, mea(y_cv, preds)))

                # Get best score
                best_params = min(score, key=itemgetter(2))[0:2]
        
    elif method == 'rf':
        if not type(params) == dict:
            raise ValueError('params is not a dictionary, please support a dictionary of parameters')
        else:            
            score = []
            end = np.floor(0.8 * X.shape[0]).astype(int)
            X_train = np.array(X)[:end]
            y_train = np.array(y)[:end]
            X_cv = np.array(X)[end:]
            y_cv = np.array(y)[end:]

            # Random search for RF
            for step in tqdm(range(steps)):
                n_estimators = np.random.randint(params['n_estimators'][0], params['n_estimators'][1])
                max_depth = np.random.choice(params['max_depth'])
                max_features = np.random.choice(params['max_features'])
                print((n_estimators, max_depth, max_features))

                rf = RandomForestRegressor(n_estimators=n_estimators,
                                                max_depth=max_depth,
                                                max_features=max_features)
                rf.fit(X_train, y_train)
                preds = rf.predict(X_cv)                                       
                score.append((n_estimators, max_depth, max_features, np.sqrt(mean_squared_error(y_cv, preds)))
                
                with open('rf_params.txt', 'w') as f:  # Use file to refer to the file object
                    f.write(str(score))

            # Get best score
            best_params = min(score, key=itemgetter(3))[0:3]
            
    elif method == 'xgboost':
        if not type(params) == dict:
            raise ValueError('params is not a dictionary, please support a dictionary of parameters')
        else:
            score = []
            end = np.floor(0.8 * X.shape[0]).astype(int)
            X_train = np.array(X)[:end]
            y_train = np.array(y)[:end]
            X_cv = np.array(X)[end:]
            y_cv = np.array(y)[end:]

            fit_params = {
                'eval_metric': 'rmse',
                'early_stopping_rounds': 10,    
                'eval_set': [(X_cv, y_cv)]
            }
            # Random search for xgboost
            for step in tqdm(range(steps)):
                n_estimators = int(np.floor(np.random.uniform(params['n_estimators'][0], params['n_estimators'][1])))
                max_depth = np.random.choice(params['max_depth'])
                lr = np.random.choice(params['learning_rate'])
                subsample = np.random.choice(params['subsample'])
                colsample_bytree = np.random.choice(params['colsample_bytree'])
                colsample_bylevel = np.random.choice(params['colsample_bylevel'])
                reg_lambda = np.random.choice(params['reg_lambda'])
                
                print((lr, n_estimators, max_depth, subsample, colsample_bytree, colsample_bylevel, reg_lambda))
                # Train & predict
                xgb_model = xgb.XGBRegressor(learning_rate=lr, 
                                             n_estimators=n_estimators,
                                             max_depth=max_depth,
                                             subsample=subsample,
                                             colsample_bytree=colsample_bytree,
                                             colsample_bylevel=colsample_bylevel,
                                             objective='reg:squarederror')

                xgb_model.fit(X_train, y_train, eval_metric=fit_params['eval_metric'],
                              early_stopping_rounds=fit_params['early_stopping_rounds'], 
                              eval_set=fit_params['eval_set'], 
                              verbose=False)
                
                preds = xgb_model.predict(X_cv)                                       
                score.append((lr, n_estimators, max_depth, subsample, colsample_bytree, colsample_bylevel, reg_lambda, \
                              np.sqrt(mean_squared_error(y_cv, preds))))
                with open('xgb_params.txt', 'w') as f:  # Use file to refer to the file object
                    f.write(str(score))

            # Get best score
            best_params = min(score, key=itemgetter(7))[0:7]
            
    elif method == 'stack':
        pass
    
    return best_params

In [153]:
# XGBoost
params = {
        'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.0005, 0.001, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        'colsample_bylevel': [0.3, 0.5, 0.6, 0.7, 0.8],
        'reg_lambda' : [0.1, 0.5, 1.0, 2, 3, 4, 5],
        'n_estimators': [1000, 10000]
}

ind = np.random.randint(0, X_train.shape[0], size=50000)
xgb_params = parameter_search(X_train.iloc[ind], y_train.iloc[ind], method='xgboost', params=params, steps=25)













  0%|          | 0/25 [00:00<?, ?it/s]

(0.05, 8313, 4, 0.9, 0.3, 0.8, 4.0)














  4%|▍         | 1/25 [00:13<05:35, 13.96s/it]

(0.05, 7034, 6, 0.8, 0.3, 0.8, 4.0)














  8%|▊         | 2/25 [00:32<05:53, 15.35s/it]

...













 92%|█████████▏| 23/25 [24:28<01:38, 49.05s/it]

(0.1, 8109, 5, 0.8, 0.4, 0.6, 4.0)














 96%|█████████▌| 24/25 [24:40<00:37, 37.99s/it]

(0.05, 5620, 6, 0.8, 0.3, 0.8, 0.5)














100%|██████████| 25/25 [24:59<00:00, 59.97s/it]


In [154]:
# RF
params = {
        'max_depth': [3, 4, 5, 6, 7],
        'n_estimators': [5000, 15000],
        'max_features': [0.8, 0.9, 1]
}

ind = np.random.randint(0, X_train.shape[0], size=50000)
rf_params = parameter_search(X_train.iloc[ind], y_train.iloc[ind], method='rf', params=params, steps=25)













  0%|          | 0/25 [00:00<?, ?it/s]

(11276, 3, 0.8)














  4%|▍         | 1/25 [09:06<3:38:42, 546.77s/it]

(12516, 7, 0.9)














  8%|▊         | 2/25 [32:25<5:07:34, 802.38s/it]

...













 76%|███████▌  | 19/25 [4:26:57<1:41:55, 1019.23s/it]

(11878, 5, 0.9)














 80%|████████  | 20/25 [4:43:49<1:24:44, 1016.87s/it]

(10247, 6, 1.0)


KeyboardInterrupt: 

In [155]:
!cat rf_params.txt

[(11276, 3, 0.8, 105984347131513.23), (12516, 7, 0.9, 106611125000613.62), (10089, 4, 0.8, 106720242970051.38), (6339, 4, 1.0, 106966354555408.44), (5946, 7, 0.8, 106587468605780.5), (7111, 3, 1.0, 106100360781854.52), (5695, 4, 1.0, 106966153782202.88), (7575, 4, 0.8, 106713771579303.81), (14387, 4, 0.9, 106884761777750.81), (13548, 7, 0.9, 106623630618127.55), (10178, 6, 0.8, 106772099451385.69), (9092, 4, 0.9, 106886357474544.6), (10118, 4, 0.8, 106699549503779.0), (7017, 3, 0.8, 105990491968806.1), (13526, 7, 1.0, 106602602807848.08), (12612, 7, 0.8, 106597139096103.22), (9879, 4, 0.9, 106897945846705.5), (12721, 6, 1.0, 106741511673449.66), (9984, 6, 0.8, 106766019153878.28), (11878, 5, 0.9, 106878401971155.92)]

In [156]:
!cat xgb_params.txt

[(0.05, 8313, 4, 0.9, 0.3, 0.8, 4.0, 1294.4494326897213), (0.05, 7034, 6, 0.8, 0.3, 0.8, 4.0, 1231.8271041217824), (0.05, 8037, 3, 0.9, 0.3, 0.7, 1.0, 1350.770670555929), (0.001, 5524, 7, 0.8, 0.3, 0.3, 3.0, 1794.914532281169), (0.0005, 2952, 4, 0.8, 0.7, 0.8, 4.0, 2198.5956608254914), (0.1, 4606, 5, 1.0, 0.3, 0.6, 1.0, 1281.2823536281846), (0.001, 5180, 3, 0.8, 0.4, 0.8, 4.0, 1469.937803715069), (0.1, 9988, 5, 0.9, 0.6, 0.5, 0.5, 1204.9175781752087), (0.05, 8997, 4, 0.8, 0.8, 0.6, 1.0, 1241.266987807888), (0.1, 8210, 4, 0.9, 0.3, 0.5, 0.1, 1283.0121311223315), (0.0005, 1262, 5, 0.9, 0.6, 0.7, 2.0, 3909.3905840954317), (0.0005, 9459, 7, 1.0, 0.8, 0.8, 0.5, 1281.268999685604), (0.1, 4421, 5, 0.8, 0.4, 0.5, 5.0, 1286.2693487607633), (0.0005, 1342, 4, 1.0, 0.6, 0.8, 5.0, 3807.9185024669796), (0.1, 1767, 7, 0.8, 0.7, 0.6, 1.0, 1178.1237738961388), (0.05, 7530, 4, 0.9, 0.7, 0.5, 2.0, 1254.179756090115), (0.05, 9615, 6, 0.9, 0.6, 0.7, 0.5, 1186.299033532866), (0.1, 5074, 6, 1.0, 0.3, 0.8, 1.

In [18]:
X_train.isnull().sum()

index                        0
Customers                    0
Year                         0
Quarter                      0
Month                        0
Week                         0
Day                          0
Store                        0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoDuration                0
CompetitionActive            0
CompetitionDays              0
RunningAnyPromo              0
RunningPromo2                0
CompetitionIntensity         0
Rel                          0
ExpectedSales                0
('0.0',)                     0
('1.0',)                     0
('not_given',)               0
('0',)                       0
('a',)                       0
('b',)                       0
('c',)                       0
('not_given',).1             0
('0.0',).1                   0
('1.0',).1                   0
('not_given',).2             0
('a',).1

In [ ]:
pca = PCA(n_components=9)
X_train = pca.fit_transform(X_train)

In [ ]:
# RF
params = {
        'ratio': np.linspace(0.01, 1, 100),
        'alpha': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 10, 20, 50, 100],
        'max_features': [0.8, 0.9, 1]
}

rf_params = parameter_search(X_train, y_train, method='rf', params=params, steps=25)